In [1]:
import pandas as pd
import os
data='../data/tcga/'

In [2]:
#./firehose_get -o .Merge_methylation__humanmethylation450__jhu_usc_edu__Level_3__within_bioassay_data_set_function__data.Level_3.2016012800.0.0.tar.gz data latest

In [3]:
samples=pd.read_csv(data+'sample.tsv',sep='\t')
print(samples.shape)
samples.index.names = ['id']
sample_ids=samples.sample_submitter_id.to_dict()
sample_ids =  {v: k for k, v in sample_ids.items()}
sample_ids['TCGA-F5-6810-01Z']

(32090, 37)


0

In [4]:
#read meth data
data_path=data+'broad/stddata__2016_01_28/'
meth_files={}
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('.tar.gz') and 'data.Level_3' in file and 'FFPE' not in file:
            cohort=file.split('org_')[1].split('.Merge')[0]
            meth_files[cohort]=os.path.join(root, file)
meth_files.keys()

dict_keys(['PCPG', 'OV', 'UVM', 'READ', 'TGCT', 'KIRC', 'LGG', 'UCEC', 'COAD', 'KICH', 'BLCA', 'BRCA', 'CESC', 'KIRP', 'ACC', 'LUSC', 'KIPAN', 'STES', 'GBMLGG', 'LIHC', 'COADREAD', 'SKCM', 'LUAD', 'UCS', 'GBM', 'THYM', 'SARC', 'PAAD', 'STAD', 'THCA', 'CHOL', 'DLBC', 'LAML', 'ESCA', 'MESO', 'PRAD', 'HNSC'])

In [6]:
nr=2
for cohort,meth_file in meth_files.items():
    print (cohort)
    if cohort in ['UCS','BRCA','KIPAN','STES','GBMLGG','THCA','PRAD','HNSC']: continue
    if cohort in ['BLCA','UCEC','PCPG','CESC','LIHC','CHOL','GBMLGG','HNSC','STAD','PAAD','LUSC','MESO','LGG','LAML',
                 'COADREAD','LUAD','BRCA','KICH','UVM']:
        df=pd.read_csv(meth_file,sep='\t',index_col=0,nrows=0,skiprows=1)
        cols=df.columns.str[:16].unique()
        df=pd.read_csv(meth_file,sep='\t',index_col=0,skiprows=1,nrows=10**nr)   

    else:
        df=pd.read_csv(meth_file,sep='\t',index_col=0,nrows=0)  
        cols=df.columns.str[:16].unique()
        df=pd.read_csv(meth_file,sep='\t',index_col=0,skiprows=0,nrows=10**nr) 
    df=df.iloc[:, range(0,4*len(cols),4)]
    print(df.shape)        
    df.columns=cols
    df=df.drop('Composite Element REF')
    df=df.dropna()    
    df=df.astype(float)
    cohorts=df.T
    cols=[c for c in cohorts.columns.tolist() if str(c) != 'nan']
    cohorts=cohorts[[c for c in cols if 'cg' in c]]
    cohorts['sample_submitter_id']=cohorts.index.str[:16]
    df=cohorts.merge(samples[['sample_submitter_id','sample_type']])  
    df=df[df.sample_type=='Solid Tissue Normal']
    df.index=df.sample_submitter_id
    df.drop(['sample_submitter_id','sample_type'],1)
    print(df.shape)
    if len(df)>0:
        df.to_csv(data+'clean/'+cohort+'.csv')

PCPG
(100, 187)
(3, 78)
OV
(100, 10)
(0, 80)
UVM
(100, 80)
(0, 80)
READ
(100, 106)
(7, 77)
TGCT
(100, 156)
(0, 80)
KIRC
(100, 480)
(152, 74)
LGG
(100, 530)
(0, 76)
UCEC
(100, 478)
(33, 76)
COAD
(100, 335)
(38, 74)
KICH
(100, 66)
(0, 77)
BLCA
(100, 434)
(21, 76)
BRCA
CESC
(100, 312)
(3, 75)
KIRP
(100, 321)
(43, 78)
ACC
(100, 80)
(0, 80)
LUSC
(100, 412)
(42, 77)
KIPAN
STES
GBMLGG
LIHC
(100, 429)
(49, 71)
COADREAD
(100, 441)
(45, 73)
SKCM
(100, 475)
(2, 77)
LUAD
(100, 492)
(31, 73)
UCS
GBM
(100, 155)
(1, 77)
THYM
(100, 126)
(1, 79)
SARC
(100, 269)
(4, 74)
PAAD
(100, 195)
(10, 76)
STAD
(100, 397)
(2, 76)
THCA
CHOL
(100, 45)
(9, 76)
DLBC
(100, 48)
(0, 80)
LAML
(100, 194)
(0, 80)
ESCA
(100, 202)
(16, 77)
MESO
(100, 87)
(0, 80)
PRAD
HNSC
